In [2]:
import numpy as np
import scipy as sp
import math
import random as rd
import pandas as pd
import torch
import json

In [3]:
dynamic = pd.read_csv("processed_dynamic.csv")
dynamic

,Unnamed: 0,stay_id,charttime,calcium,creatinine,glucose,sodium,chloride,hemoglobin,wbc,...,glucose_grad,sodium_grad,chloride_grad,hemoglobin_grad,wbc_grad,alt_grad,ast_grad,alp_grad,bilirubin_total_grad,pt_grad
0,0,30004144,2126-04-04 09:30:00,8.0,0.7,114.0,135.0,103.0,13.00,5.10,...,0.001466,0.000000,-0.000077,-0.000093,-0.000023,-0.001219,0.000008,0.008603,-7.716049e-06,0.000093
1,1,30004144,2126-04-04 13:06:00,6.8,0.7,133.0,135.0,102.0,11.80,4.80,...,0.001466,0.000000,-0.000077,-0.000093,-0.000023,-0.001219,0.000008,0.008603,-7.716049e-06,0.000093
2,2,30005366,2202-12-28 02:10:00,8.8,6.7,41.0,139.0,100.0,9.20,9.40,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
3,3,30006983,2159-10-11 13:20:00,8.0,0.8,156.0,132.0,102.0,10.10,8.20,...,-0.001230,0.000073,0.000110,-0.000015,0.000022,0.000055,0.000275,-0.000606,3.671072e-06,0.000031
4,4,30006983,2159-10-12 04:28:00,7.1,1.0,89.0,136.0,108.0,9.31,9.41,...,-0.001230,0.000073,0.000110,-0.000015,0.000022,0.000055,0.000275,-0.000606,3.671072e-06,0.000031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6984,6984,39985110,2141-03-03 12:37:00,10.4,6.8,149.0,139.0,98.0,12.60,10.30,...,0.000425,0.000018,0.000000,0.000007,0.000052,0.001016,0.002103,0.000591,-1.847746e-07,0.000019
6985,6985,39986206,2183-06-19 23:56:00,7.6,5.6,121.0,139.0,103.0,10.00,19.20,...,-0.000866,0.000000,0.000000,0.000017,-0.000039,0.004264,0.001771,0.002714,5.627706e-06,0.000013
6986,6986,39986206,2183-06-20 06:21:00,7.5,6.0,101.0,139.0,103.0,10.40,18.30,...,-0.000866,0.000000,0.000000,0.000017,-0.000039,0.004264,0.001771,0.002714,5.627706e-06,0.000013
6987,6987,39992167,2114-06-10 21:12:00,7.9,0.9,113.0,131.0,100.0,6.80,6.00,...,0.001674,-0.000218,0.000000,0.000000,-0.000102,-0.007358,-0.007773,-0.010953,1.310044e-04,0.000022


In [4]:
index = dynamic["stay_id"]
index
patients = set()
for pat in index:
    patients.add(pat)
patents = tuple(patients)
records = []
m = 0
for patient in patients:
    ts = dynamic[dynamic["stay_id"]==patient]
    ts = ts.drop(["Unnamed: 0","stay_id","charttime"], axis=1)
    if len(ts.index) > m:
        m = len(ts.index)
    records.append(ts.values)
print(records[0])
print(m)

[[ 8.90000000e+00  1.00000000e+00  1.20000000e+02  1.40000000e+02
   1.03000000e+02  1.54000000e+01  5.60000000e+00  4.34000000e+01
   9.95000000e+01  2.57000000e+02  1.84000000e+00  1.54500000e+01
   1.00000000e+00  0.00000000e+00  0.00000000e+00  5.55555556e-03
   3.70370370e-04 -7.40740741e-04 -7.40740741e-05  3.70370370e-05
   1.33333333e-03 -1.07407407e-02  4.63333333e-02 -7.40740741e-06
  -1.90740741e-03]
 [ 9.30000000e+00  1.10000000e+00  1.03000000e+02  1.42000000e+02
   1.04000000e+02  1.67000000e+01  8.00000000e+00  1.02500000e+02
   1.43400000e+02  1.08100000e+02  1.05000000e+00  1.54500000e+01
   1.00000000e+00  0.00000000e+00  0.00000000e+00  5.55555556e-03
   3.70370370e-04 -7.40740741e-04 -7.40740741e-05  3.70370370e-05
   1.33333333e-03 -1.07407407e-02  4.63333333e-02 -7.40740741e-06
  -1.90740741e-03]
 [ 9.30000000e+00  1.10000000e+00  1.18000000e+02  1.43000000e+02
   1.02000000e+02  1.65000000e+01  8.10000000e+00  1.06100000e+02
   1.14400000e+02  2.33200000e+02  1.0

In [5]:
def padding_zero(data: np.ndarray, size: int):
    cols = data.shape[0]
    if size <= cols:
        return torch.from_numpy(data)
    ans = np.zeros((size-cols,data.shape[1]))
    res = np.concatenate((data,ans),axis = 0)
    return torch.from_numpy(res)
    

In [6]:
def padding_average(data: np.ndarray, size: int):
    cols = data.shape[0]
    if size <= cols:
        return torch.from_numpy(data)
    ans = data.mean(axis=0)
    for i in range(size-cols):
      data = np.concatenate((data,ans),axis = 0)
    return torch.from_numpy(data)

In [11]:
for i in range(len(records)):
    records[i] = padding_zero(records[i], m)
tmp = tuple(records)
print(type(tmp[0]))
input_data = torch.stack(tmp)
input_data.shape

<class 'torch.Tensor'>


torch.Size([1923, 196, 25])

In [12]:
static = pd.read_csv('processed_static.csv')
static.columns
label = static[["stay_id", "vent_duration"]]
input_label = []
for patient in patients:
    dur = label[label["stay_id"]==patient]
    input_label.append(dur.values.flatten()[1])
# input_label = torch.tensor(input_label)
# input_label
# torch.save(input_data,"input_data_averaged.pt")
# torch.save(input_label,"input_label.pt")

In [14]:
static

,stay_id,vent_duration
0,30004144,25.000000
1,30005366,6.000000
2,30006983,49.000000
3,30023204,24.166667
4,30031418,9.900000
...,...,...
1918,39977971,4.000000
1919,39982332,13.000000
1920,39985110,31.266667
1921,39986206,222.000000
